# Parsing and Recombining Inputs

In the chapter on [Grammars](Grammars.ipynb), we discussed how grammars can be
used to represent various languages. We also saw how grammars can be used to
generate strings of the corresponding language. Grammars can also perform the
reverse. That is, given a string, one can decompose the string into its
constituent parts that correspond to the parts grammar used to generate it
-- the derivation tree of that string. These parts (and parts from other similar
strings) can later be recombined using the same grammar to produce new strings.

In this chapter, we use grammars to parse and decompose inputs to
their corresponding derivation trees, allowing us to recombine them
arbitrarily.

**Prerequisites**

* You should have read the [chapter on grammars](Grammars.ipynb).
* An understanding of derivation trees from the [chapter on grammar fuzzer](GrammarFuzzer.ipynb)
  is also required.

In [ ]:
import fuzzingbook_utils
from Grammars import EXPR_GRAMMAR, START_SYMBOL, RE_NONTERMINAL
from GrammarFuzzer import display_tree, all_terminals

In order to parse a string, one needs to identify the language, and the
corresponding grammar. For example, here is a string that we would like to parse

In [ ]:
mystring = '1+2'

This string is an arithmetic expression for addition, which may be specified using a grammar.

In [ ]:
A1_GRAMMAR = {
   "<start>":
       ["<expr>"],
   "<expr>":
       ["<expr>+<expr>", "<expr>-<expr>", "<integer>"],
   "<integer>":
       ["<digit><integer>", "<digit>"],
   "<digit>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

The parse tree for our expression from this grammar is given by:

In [ ]:
tree = ('<start>',[
    ('<expr>',[
        ('<expr>',[('<integer>',[('<digit>',[('1',[])])])]),
        ('+',[]),                           
        ('<expr>',[('<integer>',[('<digit>',[('2',[])])])])])])
assert mystring == all_terminals(tree)
display_tree(tree)

While a grammar can be used to specify a given language, there could be multiple
grammars that correspond to the same language. For example, here is another 
grammar to describe the same addition expression.

In [ ]:
A2_GRAMMAR = {
   "<start>":
      ["<expr>"],
   "<expr>":
      ["<integer><expr_>"],
   "<expr_>":
      ["+<expr>", "-<expr>", ""],
   "<integer>":
      ["<digit><integer_>"],
   "<integer_>":
      ["<integer>", ""],
   "<digit>":
      ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

The corresponding derivation tree is given by:

In [ ]:
tree = ('<start>', [
    ('<expr>', [('<integer>',
                 [('<digit>', [('1', [])]), ('<integer_>', [('',[])])]),
                ('<expr_>', [('+', []), 
                             ('<expr>', [
                                 ('<integer>', [
                                     ('<digit>', [('2', [])]),
                                     ('<integer_>', [('',[])])]),
                                 ('<expr_>', [('',[])])])])])])
assert mystring == all_terminals(tree)
display_tree(tree)

Indeed, there could be different classes of grammars that
describe the same language. For example, the first grammar `A1_GRAMMAR`
is a grammar that sports both _right_ and _left_ recursion, while the
second grammar `A2_GRAMMAR` does not have left recursion in the
non-terminals in any of its productions, but contains _epsilon_ productions.
(An epsilon production is a production that has empty string in its right
hand side.)

A grammar is left recursive if any of its non-terminals are left recursive,
and a non-terminal is directly left-recursive if the left-most symbol of
any of its productions is itself. It is indirectly left-recursive if any
of the left-most symbols can be expanded using their definitions to
produce the non-terminal as the left-most symbol of the expansion.
Right recursive grammars are defined similarly.

For example, in `A1_GRAMMAR`, the definition of `<expr>` is
left-recursive, and right recursive directly. However in `A2_GRAMMAR`,
`<expr>` is right recursive indirectly through the expansion of `<expr_>`.

To complicate matters further, there could be
multiple derivation trees -- also called _parses_ -- corresponding to the
same string from the same grammar. For example, a string `1+2+3` can be parsed
in two ways as we see below using the `A1_GRAMMAR`

In [ ]:
mystring = '1+2+3'
tree = ('<start>', [
    ('<expr>', [
        ('<expr>', [
            ('<expr>', [('<integer>', [('<digit>', [('1', [])])])]),
            ('+', []),
            ('<expr>', [('<integer>', [('<digit>', [('2', [])])])])]), 
        ('+', []), 
        ('<expr>', [('<integer>', [('<digit>', [('3', [])])])])])])
assert mystring == all_terminals(tree)
display_tree(tree)

In [ ]:
tree = ('<start>', [
    ('<expr>', [
        ('<expr>', [('<integer>', [('<digit>', [('1', [])])])]), 
        ('+', []), 
        ('<expr>', [
            ('<expr>', [('<integer>', [('<digit>', [('2', [])])])]), 
            ('+', []), 
            ('<expr>', [('<integer>', [('<digit>', [('3', [])])])])]) ])])
assert all_terminals(tree) == mystring
display_tree(tree)

Numerous parsing techniques exist that can take a grammar and the given
string, and produce the corresponding derivation tree or trees. However,
some of these techniques work only on specific classes of grammars.
These classes of grammars are named after the specific kind of parser
that can accept grammars of that category. That is, the upperbound for
the capabilities of the parser defines the grammar class named after that
parser.

Different classes of grammars differ in the features that are available to
the user for writing a grammar of that class. That is, the corresponding
kind of parser will be unable to parse a grammar that makes use of more
features than is allowed. For example, the `A2_GRAMMAR` is an _LL(1)_
grammar because it lacks left recursion, while `A1_GRAMMAR` is not an
_LL(1)_ grammar. This is because an _LL_ parser is a parser that parses
its input from left to right, and constructs a leftmost derivation of its
input by expanding the non-terminals it encounters. If there is a left
recursion in one of these rules, an _LL_ parser will enter an infinite loop.

We will examine a few classes of parsers next. First we define a few common
features required by all parsers we implement.

In [ ]:
import functools
import re

The  `EXPR_GRAMMAR` we import from the [chapter on grammars](Grammars.ipynb) is oriented towards generation. In particular, the production rules are stored as strings. We need to massage this representation a little to conform to a canonical representation where each token in a rule is represented separately.

In [ ]:
def split(rule):
    return [s for s in re.split(RE_NONTERMINAL, rule) if s]

def canonical(grammar):
    return  {k: [split(l) for l in rules] for k, rules in grammar.items()}

We define a minimal interface for parsing, that is obeyed by all parsers.

In [ ]:
class Parser(object):
    def __init__(self, grammar, start_symbol=START_SYMBOL):
        self.start_symbol = start_symbol
        self.grammar = grammar
        
    def parse_prefix(self, text):
        """Return pair (cursor, forest) for longest prefix of text"""
        raise NotImplemented()
        
    def parse(self, text):
        cursor, forest = self.parse_prefix(text)
        if cursor < len(text):
            raise SyntaxError("at " + repr(text[cursor:]))
        return forest

## Packrat Parsing for _Parsing Expression Grammars_

Short of handrolling a parser, _Packrat_ parsing is one of the simplest parsing techniques, and the class of grammar it accepts is called a _Parsing Expression Grammar_ (Packrat is one of the techniques for parsing PEGs).

A _Parsing Expression Grammar_ comes is very similar to a _Context-Free-Grammar_. As in a _Context Free Grammar_, the grammar is represented by a set of non-terminals and corresponding **choice expressions** representing how to match each. For example, here is a PEG that matches `a` or `b`.

In [ ]:
peg = {
    '<start>': [['a'],['b']]
}

However, unlike the _CFG_, the choice expression is an **ordered choice**. That is, rather than choosing all rules that can potentially match, we stop at the first match that succeed. For example, the below _PEG_ can match `ab` but not `abc` unlike a _CFG_ which will match both.

In [ ]:
peg = {
    '<start>': [['ab'],['abc']]
}

Each choice in a _choice expression_ represents a rule on how to satisfy that particular choice. The choice is a sequence of tokens (terminals and non-terminals) that are matched against a given text as in a _CFG_.

Beyond the syntax of grammar definitions we have seen so far, a _PEG_ can can also contain a few additional elements. See the exercises at the end of the chapter for additional information.

The parsing expression grammars model the typical practice in handwritten recursive descent parsers, and hence it may be considered more intuitive to understand. Further, it comes with attractive properties such as linear time parsing (the time required to parse increases linearly with the increase in input length).

We look at the implementation of a simple PEG parser next.

We derive from the `Parser` base class first, and we accept the text to be parsed in the `parse` method, which in turn calls `unify_key` with the `start_symbol` which is the starting point of parsing.

In [ ]:
class PEGParser(Parser):
    def __init__(self, grammar, start_symbol):
        super().__init__(canonical(grammar), start_symbol)
        
    def parse_prefix(self, text):
        return self.unify_key(self.start_symbol, text, 0)

The `unify_key` algorithm is simple. If given a terminal symbol, it tries to match the symbol with the current position in the text. If the symbol and text match, it returns successfully with the new parse index `at`.

If on the other hand, it was given a non-terminal, it retrieves the choice expression corresponding to the key, and tries to match each choice in order using `unify_rule`. If **any** of the rules succeed in being unified with the given text, the parse is considered a success, and we return with the new parse index returned by `unify_rule`.

In [ ]:
class PEGParser(PEGParser):
    @functools.lru_cache(maxsize=None)
    def unify_key(self, key, text, at=0):
        if key not in self.grammar:
            if text[at:].startswith(key): return at + len(key), (key, [])
            else: return at, None
        for rule in self.grammar[key]:
            l, res = self.unify_rule(rule, text, at)
            if res: return (l, (key, res))
        return 0, None

The `unify_rule` is similar. It retrieves the tokens corresponding to the rule that it needs to unify with the text, and calls `unify_key` on them in sequence. If **all** tokens are successfully unified with the text, the parse is a success.

In [ ]:
class PEGParser(PEGParser):
    def unify_rule(self, rule, text, at):
        results = []
        for token in rule:
            at, res = self.unify_key(token, text, at)
            if res is None: return at, None
            results.append(res)
        return at, results

The two methods are mutually recursive, and given that `unify_key` tries each alternative until it succeeds, `unify_key` can be called multiple times with the same arguments. Hence, we memoize the results of `unify_key` with `functools.lru_cache` decorator. This memoization gives the algorithm its name -- packrat.

We wrap initialization and calling of PEGParser in a method `parse` that accepts the text to be parsed along with the grammar.

In [ ]:
def parse(text, grammar):
    peg = PEGParser(grammar, START_SYMBOL)
    return peg.parse(text)  

Here are a few examples of our parser in action.

In [ ]:
mystring = "1 + (2 * 3)"
tree = parse(mystring, EXPR_GRAMMAR)
assert all_terminals(tree) == mystring
display_tree(tree)

In [ ]:
mystring = "1 * (2 + 3.35)"
tree = parse(mystring, EXPR_GRAMMAR)
assert all_terminals(tree) == mystring
display_tree(tree)

One should be aware that while the grammar looks like a Context Free Grammar, the language described by a PEG may be different (only LL(1) grammars are guaranteed to represent the same language for both PEGs and other parsers), while other behaviors could be surprising~\cite{redziejowski2008}. 

While _Parsing Expression Grammars_ are simple at first sight, their behavior in some cases might be a bit unintuitive. For example, here is an example from Redziejowski~\cite{redziejowski}

In [ ]:
PEG_SURPRISE = {
    "A": ["a<A>a","aa"]
}

When interpreted as a context free grammar and used as a string generator, it will produce strings of the form `a, aa, aaaa, aaaaaa` that is, it produces strings where the number of `a` is \Latex{2*n}. However, the PEG can only recognize strings of the form `a, aa, aaaa, aaaaaaaa`, that is where the number of `a` is \Latex{2^n}.

## Parsers for Context Free Grammars

While the _Parsing Expression Grammars_ are expressive and the _packrat_ parser for parsing them is simple and intuitive, PEGs suffer from a major deficiency for our purposes. PEGs are oriented towards language recognition, and it is not clear how to translate an arbitrary PEG to a _CFG_. As we mentioned earlier, a naive re-interpretation of a PEG as a CFG does not work very well. Further, it is not clear what is the exact relation between the class of languages represented by _PEG_ and the the class of languages represented by _CFG_. Since our primary focus is _fuzzing_ -- that is _generation_ of strings, we next look at parsers that can accept _CFG_.

We require meta characters for marking an empty string, and for marking the end of input.

In [ ]:
EOF = '\0'
EPSILON = ''

The `rules` method takes in a grammar and returns `(key,production)` pairs for each production in the grammar. This is an alternative representation of a grammar.

In [ ]:
def rules(g): return [(k, e) for k, a in g.items() for e in a]

In [ ]:
rules(canonical(EXPR_GRAMMAR))

The `terminals` method extracts all terminal symbols from a `canonical` grammar representation.

In [ ]:
def terminals(g):
    return set(t for k, expr in rules(g) for t in expr if t not in g)

In [ ]:
terminals(canonical(EXPR_GRAMMAR))

While parsing, one often requires to know whether a given non-terminal can derive an empty string. For example, in the following grammar `A` can derive an empty string, while `B` can not. The non-terminals that can derive an empty string is called a `nullable` non-terminal.

In [ ]:
my_grammar = {
    '<start>': [['<A>'], ['<B>']],
    '<A>': [['a'],['']],
    '<B>': [['b']]
}

Computing the `nullable` set requires expanding each production rule in the grammar iteratively and inspecting whether a given rule can derive the empty string. Each iteration needs to take into account new terminals that have been found to be nullable. The procedure stops when we obtain a stable result. This procedure can be abstracted into a more general method `fixpoint`. A `fixpoint` of a function is an element in the function's domain such that it is mapped to itself. For example, `1` is a `fixpoint` of square root because `squareroot(1) == 1`. 

(We use `str` rather than `hash` to check for equality in `fixpoint` because `set`, which we would like to use as an argument has a good string representation but is not hashable).

In [ ]:
def fixpoint(f):
    def helper(arg):
        while True:
            sarg = str(arg)
            arg_ = f(arg)
            if str(arg_) == sarg:
                return arg
            arg = arg_
    return helper

Remember `my_sqrt` from the first chapter? We can define `my_sqrt` using `fixpoint`.

In [ ]:
def my_sqrt(x):
    @fixpoint
    def _my_sqrt(approx):
        return (approx + x / approx) / 2
    return _my_sqrt(1)

In [ ]:
my_sqrt(2)

Similarly, we can define `nullable` using `fixpoint`. We essentially provide the definition of a single intermediate step. That is, assuming that `nullables` contain the current nullable non-terminals, we iterate over the grammar looking for productions which are `nullable` -- that is, productions where the entire sequence can yield an empty string on some expansion.

In [ ]:
def nullable(grammar):
    productions = rules(grammar)
    @fixpoint
    def nullable_(nullables):
        for A, expr in productions:
            if all(token in nullables for token in expr):
                nullables |= {A}
        return (nullables)
    return nullable_({EPSILON})

The nullable non-terminals in `my_grammar` is computed as expected (It includes EPSILON which is technically not a non-terminal, but you can ignore that for now).

In [ ]:
nullable(my_grammar)

We require an additional method to massage our `EXPR_GRAMMAR`. This method removes the extraneous spaces from the grammar definition. While the spaces are reasonable when one is generating strings, they make our parses needlessly complicafed.

In [ ]:
def shrink(rule): return [i.strip() for i in rule]
def canonical(grammar):
    return  {k: [shrink(split(l)) for l in rules] for k, rules in grammar.items()}

In [ ]:
canonical(EXPR_GRAMMAR)

### Earley parser

The Earley parser is a general parser that is able to parse arbitrary _Context Free Grammars_.

In [ ]:
class State(object):
    def __init__(self, name, expr, dot, origin, children=[]):
        self.name, self.expr, self.dot, self.origin = name, expr, dot, origin
        self.children = children[:]
    def finished(self): return self.dot >= len(self.expr)
    def shift(self):
        return State(self.name, self.expr, self.dot+1, self.origin, self.children)
    def symbol(self): return self.expr[self.dot]

    def _t(self): return (self.name, self.expr, self.dot, self.origin.i, tuple(self.children))
    def __hash__(self): return hash(self._t())
    def __eq__(self, other): return  self._t() == other._t()

class Column(object):
    def __init__(self, i, token):
        self.token, self.states, self._unique, self.i = token, [], {}, i

    def add(self, state):
        if state in self._unique: return self._unique[state]
        self._unique[state] = state
        self.states.append(state)
        return self._unique[state]

class EarleyParser(Parser):
    # http://courses.washington.edu/ling571/ling571_fall_2010/slides/parsing_earley.pdf
    # https://github.com/tomerfiliba/tau/blob/master/earley3.py
    def _parse(self, words, start):
        # Aycock 2002 Practical Earley Parsing -- treatment of epsilon
        epsilon = nullable(self.grammar)
        alt = tuple(*self.grammar[start])
        chart = [Column(i, tok) for i,tok in enumerate([None, *words])]
        chart[0].add(State(start, alt, 0, chart[0], []))

        for i, col in enumerate(chart):
            for state in col.states:
                if state.finished():
                    self.complete(col, state)
                else:
                    sym = state.symbol()
                    if sym in self.grammar:
                        self.predict(col, sym)
                        if sym in epsilon:
                            # note that precomputation of epsilon derivation can result in infinite
                            # loops for certain grammars. Hence, we mark a nullable non-terminal
                            # but do not expand it.
                            col.add(state.shift()).children.append(State(sym + '*', tuple(), 0, col))
                    else:
                        if i + 1 >= len(chart): continue
                        self.scan(chart[i+1], state, sym)
        return chart
    
    def predict(self, col, sym):
        for alt in self.grammar[sym]:
            col.add(State(sym, tuple(alt), 0, col))

    def scan(self, col, state, token):
        if token == col.token:
            col.add(state.shift())

    def complete(self, col, state):
        for st in state.origin.states:
            if st.finished(): continue
            if state.name != st.symbol(): continue
            col.add(st.shift()).children.append(state)
        

    def parse(self, text):
        table = self._parse(text, self.start_symbol)
        states = [st for st in table[-1].states if st.name == '<start>' and st.finished()]
        for state in states:
            yield self.node_translator(state)
            
    def process_expr(self, expr, children):
        terms = iter([(i,[]) for i in expr if i not in self.grammar])
        nts = iter([self.node_translator(i) for i in  children])
        return [next(terms if i not in self.grammar else nts) for i in expr]

    def node_translator(self, state):
        return (state.name, self.process_expr(state.expr, state.children))


In [ ]:
def parse(text, grammar, start=START_SYMBOL):
    ep = EarleyParser(grammar, start)
    return ep.parse(text)  

In [ ]:
new_grammar = canonical(EXPR_GRAMMAR)
for tree in parse(list('1+2+3'), new_grammar):
    display_tree(tree)

#### Ambiguous grammars generates parse forests

In [ ]:
grammar= {
        '<start>': ['<A>'],
        '<A>': ['<A>+<A>', 'a'],
        }

In [ ]:
new_grammar = canonical(grammar)
for tree in parse(list('a+a+a'), new_grammar, '<start>'):
    display_tree(tree)

## Further Information
* Parsing with arbitrary context free grammars is reducible to boolean matrix multiplication~\cite{Valiant1975} (and the reverse~\cite{Lee2002}) which is at present bounded by \Latex{O(2^{23728639}}) ~\cite{LeGall2014}
* The actual class of languages that is expressible in PEG is currently unknown. In particular, we know that PEGs can express certain languages such as \Latex{a^n b^n c^n}. However, we do not know if there exist Context-Free languages that are not expressible with PEGs.

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Exercises

Close the chapter with a few exercises such that people have things to do.  In Jupyter Notebook, use the `exercise2` nbextension to add solutions that can be interactively viewed or hidden:

* Mark the _last_ cell of the exercise (this should be a _text_ cell) as well as _all_ cells of the solution.  (Use the `rubberband` nbextension and use Shift+Drag to mark multiple cells.)
* Click on the `solution` button at the top.

(Alternatively, just copy the exercise and solution cells below with their metadata.)

### Exercise 1

The _PEG_ syntax provides a few notational conveniences reminiscent of regular expressions. For example, it supports the following operators (letters `T` and `A` represents tokens that can be either terminal or non-terminal. `ε` is an empty string, and `/` is the ordered choice operator similar to the non-ordered choice operator `|`):

* `T?` represents an optional greedy match of T and `A := T?` is equivalent to `A := T/ε`.
* `T*` represents zero or more greedy matches of `T` and `A := T*` is equivalent to `A := T A/ε`.
* `T+` represents one or more greedy matches -- equivalent to `TT*`

If you look at the three notations above, each can be represented in the grammar in terms of basic syntax.
Remember the exercise from [the chapter on grammars](Gramamr.ipynb) that developed `define_ex_grammar` that can represent grammars as Python code? extend `define_ex_grammar` to `define_peg` to support the above notational conveniences. The decorator should rewrite a given grammar that contain these notations to an equivalent grammar in basic syntax.

_No solution yet_

In [ ]:
pass

_Some more text for the solution_

### Exercise 2

Beyond these notational conveniences, it also supports two predicates that can provide a powerful lookahead facility that does not consume any input.

* `T&A` represents an _And-predicate_ that matches `T` if `T` is matched, and it is immediately followed by `A`
* `T!A` represents a _Not-predicate_ that matches `T` if `T` is matched, and it is *not* immediately followed by `A`

Implement these predicates in our _PEG_ parser.

_No solution yet_

In [ ]:
pass